In [ ]:
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime
import json
import os

# Funções utilitárias definidas diretamente no notebook
def load_settings(settings_file='../settings.json'):
    """Carrega configurações de um arquivo JSON"""
    try:
        if os.path.exists(settings_file):
            with open(settings_file, 'r', encoding='utf-8') as f:
                settings = json.load(f)
                print(f"✅ Configurações carregadas de: {settings_file}")
                return settings
        else:
            print(f"⚠️ Arquivo {settings_file} não encontrado, usando configurações padrão")
            return {
                "api_url": "https://api.bcb.gov.br/dados/serie/bcdata.sjson?serie=12",
                "raw_data_path": "./data/raw/"
            }
    except Exception as e:
        print(f"❌ Erro ao carregar configurações: {e}")
        return {
            "api_url": "https://api.bcb.gov.br/dados/serie/bcdata.sjson?serie=12",
            "raw_data_path": "./data/raw/"
        }

def ensure_path_exists(path):
    """Garante que o diretório existe"""
    try:
        Path(path).mkdir(parents=True, exist_ok=True)
        print(f"✅ Diretório garantido: {path}")
        return True
    except Exception as e:
        print(f"❌ Erro ao criar diretório {path}: {e}")
        return False

# Carrega configurações
settings = load_settings()
API_URL = settings.get("api_url", "")
RAW_DATA_PATH = Path(settings.get("raw_data_path", "./data/raw/"))

# Garante que o caminho existe
ensure_path_exists(RAW_DATA_PATH)

# Função para coletar dados do BCB
def coletar_dados_bcb(api_url):
    print(f"Requisitando dados de: {api_url}")
    response = requests.get(api_url)
    if response.status_code == 200:
        print("✅ Dados recebidos com sucesso")
        return pd.DataFrame(response.json())
    else:
        raise Exception(f"❌ Erro ao acessar API: {response.status_code}")

# Verificar se API_URL não está vazio ou inválido
if not API_URL or "bcdata.sjson" in API_URL:
    print("Iniciando coleta de dados")
    API_URL = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.1783/dados?formato=json"

# Executar coleta
df_raw = coletar_dados_bcb(API_URL)

# Conversão de data e ordenação
df_raw['data'] = pd.to_datetime(df_raw['data'], dayfirst=True)
df_raw = df_raw.sort_values('data')

# Salvar dados
file_name = f'dados_bcb_{datetime.now().strftime("%Y%m%d_%H%M%S")}.parquet'
output_path = RAW_DATA_PATH / file_name
df_raw.to_parquet(output_path, index=False)

print(f"Dados salvos em: {output_path}")
print(f"Shape dos dados: {df_raw.shape}")
df_raw.head()

⚠️ Arquivo ../settings.json não encontrado, usando configurações padrão
✅ Diretório garantido: data/raw
⚠️  Usando URL correta do arquivo de configurações
📡 Requisitando dados de: https://api.bcb.gov.br/dados/serie/bcdata.sgs.1783/dados?formato=json
✅ Dados recebidos com sucesso
Dados salvos em: data/raw/dados_bcb_20250718_233936.parquet
Shape dos dados: (544, 2)


,data,valor
0,1980-02-01,174000000
1,1980-03-01,170000000
2,1980-04-01,177000000
3,1980-05-01,183000000
4,1980-06-01,195000000
